# System Details
The system-related commands are not related directly to waveform generation but are an 
important part of operating the Proteus unit. These commands are used to reset the instrument and 
query its system settings and version/release related informations.



In [9]:
#Pre-requisite for the script 
ip_address= '192.90.70.22' #input('Please enter ip address for the device') 

In [10]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

In [11]:
# Connect to instrument
inst = connect(ip_address)

# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# # Get the model:
# model_name = inst.send_scpi_query('SYST:INF:MODel?')
# print('Model: {0} '.format(model_name))
# # Get number of channels
# resp = inst.send_scpi_query(":INST:CHAN? MAX")
# #print("Number of channels: " + resp)
# num_channels = int(resp)

# # Get the DAC mode (8 bits or 16 bits)
# resp = inst.send_scpi_query(':SYST:INF:DAC?')
# if resp == 'M0':
#     dac_mode = 16
# else:
#     dac_mode = 8

# #print('DAC {0} bits'.format(dac_mode))

# if dac_mode == 16:
#     max_dac = 65535
#     data_type = np.uint16 
# else:
#     max_dac = 255
#     data_type = np.uint8 
    
# half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':TRACe:DELete:ALL')
# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

Service connect
Trying to connect to IP:192.90.70.22
Tabor Electronics,P9484M,000002232771,1.238.6 --slot#: 2, 
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes


In [12]:
## Voltage query
## 12.17 Internal Vcc of the instrument
resp = inst.send_scpi_query(":SYST:VINT?")
print('Internal Vcc of the instrument : {}V'.format(resp)  ) 

## 12.18 auxiliary Vcc of the instrument.
resp = inst.send_scpi_query(":SYST:VAUX?")
print('Auxiliary Vcc of the instrument : {}V'.format(resp)  )  
## 12.1 verbose level (0: minimal, 6: maximal)
min = int(inst.send_scpi_query(":SYST:LOG? MIN")) ## minimum value
print('Minimum verbose level = ', min)
max = int(inst.send_scpi_query(":SYST:LOG? MAX")) ## Maximum value
print('Maximum verbose level = ', max)   

Internal Vcc of the instrument : 0.967667V
Auxiliary Vcc of the instrument : 1.80482V
Minimum verbose level =  0
Maximum verbose level =  7


In [13]:
## Temperatue query
## 12.14 The Proteus unit will return the internal temperature (°C).
resp = inst.send_scpi_query(":SYST:TEMP?")
print('Temperature of the instrument : ',resp)
resp = inst.send_scpi_query(":SYST:HTP?")
print('Highest recorded temperature of the instrument (°C) : ',resp)
resp = inst.send_scpi_query(":SYST:LTP?")
print('Lowest recorded temperature of the instrument (°C) : ',resp)  
## 12.13 SCPI command version
resp = inst.send_scpi_query(":SYST:INF:SCPI:VERS?")
print('SCPI command version : ',resp)
## 12.3  last calibration date
resp = inst.send_scpi_query(":SYST:INF:CAL?")
print('Last calibration date : ',resp)

## 12.4 Model id
resp = inst.send_scpi_query(":SYST:INF:MOD?")
print('Model id : ',resp)

## 12.5 Serial number
resp = inst.send_scpi_query(":SYST:INF:SER?")
print('Serial number : ',resp)

## 12.6 hardware revisions in the form "<AB-<AM>-DB>-<DM>-<ADC>"
resp = inst.send_scpi_query(":SYST:INF:HARD?")
print('Hardware revision level : ',resp)
##12.7 Query the FPGA FW version
resp = inst.send_scpi_query(":SYST:INF:FPGA:VERS?")
print('FPGA FW version : ',resp)
##12.8 Query the FPGA FW build date
resp = inst.send_scpi_query(":SYST:INF:FPGA:DATE?")
print('FPGA FW build date : ',resp)

##12.9 Query the control PC DLL version
resp = inst.send_scpi_query(":SYST:INF:FIRM:VERS?")
print('DLL version : ',resp)

## 12.10 Proteus DLL build date
resp = inst.send_scpi_query(":SYST:INF:FIRM:DATE?")
print('DLL build date : ',resp)

## 12.11 DAC mode
resp = inst.send_scpi_query(":SYST:INF:DAC?")
##resp = 'M1'
if resp == 'M0':
    print('DAC mode : 16 bit',resp)
elif resp == 'M1':
    print('DAC mode : 8 bit')
#Returns M0 for 16-bit width and M1 for 8-bit width.

## 12.12  slot-number
resp = inst.send_scpi_query(":SYST:INF:SLOT?")
print('slot-number : ',resp) 
# 12.2 . This query will interrogate the Proteus unit for programming errors (refer programming maual for error list)
resp = inst.send_scpi_query(":SYST:ERR?")
print('ERROR : ',resp) 

Temperature of the instrument :  71.338800, 2
Highest recorded temperature of the instrument (°C) :  72.9301
Lowest recorded temperature of the instrument (°C) :  28.1295
SCPI command version :  1.129.000
Last calibration date :  2023-07-12 16:05
Model id :  P9484M
Serial number :  2232771
Hardware revision level :  E-D-E-D-XX
FPGA FW version :  1.238.6
FPGA FW build date :  2023-05-14/0
DLL version :  1.200.0-rc
DLL build date :  2024-01-04 09:17:28
DAC mode : 16 bit M0
slot-number :  2
ERROR :  0, no error


In [14]:
# voltage level
resp = inst.send_scpi_query(":VOLT?DEF")
print('Default Voltage :',resp)
max = inst.send_scpi_query(":VOLT?MAX")
min = inst.send_scpi_query(":VOLT?MIN")

print('Voltage Range {0} to {1}:'.format(min,max))

inst.send_scpi_cmd(':VOLT 0.2')
resp = inst.send_scpi_query(':VOLT?')
print('Voltage : {0}'.format(resp))

Default Voltage : 0.5
Voltage Range 0.001 to 0.55:
Voltage : 0.2


In [15]:
# offfset 
resp = inst.send_scpi_query(':VOLT:OFFS?DEF')
print('Default Voltage offset : {0}'.format(resp))
resp = inst.send_scpi_query(':VOLT:OFFS?MAX')
print('Voltage offset(max) : {0}'.format(resp))
resp = inst.send_scpi_query(':VOLT:OFFS?MIN')
print('Voltage offset(min) : {0}'.format(resp))
inst.send_scpi_cmd(':VOLT:OFFS 0.2')
resp = inst.send_scpi_query(':VOLT:OFFS?')
print('Voltage offset : {0}'.format(resp))

Default Voltage offset : 0
Voltage offset(max) : 0
Voltage offset(min) : 0
Voltage offset : 0


In [16]:
#disconnect
disconnect()
print('Proteus disconnected successfully')